In [ ]:
pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install ray[tune]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 kB 31.5 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.54.0
    Uninstalling grpcio-1.54.0:
      Successfully uninstalled grpcio-1.54.0


In [ ]:
pip install ray[all]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
pip install nevergrad

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 20.3 MB/s eta 0:00:00


In [ ]:

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers, metrics
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, LeakyReLU
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt


print(tf.config.list_physical_devices('GPU'))

[]


In [ ]:
import argparse
import os

from filelock import FileLock
from tensorflow.keras.datasets import mnist

import ray
from ray import air, tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.integration.keras import TuneReportCallback
from ray.tune.search.bayesopt import BayesOptSearch
from ray.tune.search.nevergrad import NevergradSearch
import nevergrad as ng

print(ng)



<module 'nevergrad' from '/usr/local/lib/python3.9/dist-packages/nevergrad/__init__.py'>


In [ ]:
def train_mnist(config):
    # https://github.com/tensorflow/tensorflow/issues/32159
    import tensorflow as tf
    num_classes = 10

    # batch_size = 128

    # epochs = 1

    with FileLock(os.path.expanduser("~/.data.lock")):
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
        n = 5000
        x_train  = x_train[0:n]
        y_train  = y_train[0:n]

    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
    )

    model.compile(
        loss="sparse_categorical_crossentropy",
        # optimizer=tf.keras.optimizers.Adam(learning_rate=config["lr"], beta_1=config["beta1"], beta_2= config['beta2']),
        optimizer=tf.keras.optimizers.SGD(learning_rate=config["lr"], momentum =config["beta1"] ),
        metrics= "accuracy",
    )

    #train the model
    model.fit(
        x_train,
        y_train,
        batch_size=config['batch_size'],
        epochs=config['epochs'],
        verbose=0,
        validation_data=(x_test, y_test),
        callbacks=[TuneReportCallback({ "mean_accuracy": "accuracy","mean_loss": "loss", "mean_accuracy_val": "val_accuracy"})],
        # callbacks=[TuneReportCallback({"mean_accuracy_val": "val_accuracy", "mean_accuracy": "accuracy"})],
        # {"mean_accuracy_val": "val_accuracy", "mean_accuracy": "accuracy", "mean_loss": "loss"}
    )

In [ ]:
def tune_grid(num_training_iterations):
    sched = AsyncHyperBandScheduler(
        time_attr="training_iteration", max_t=400, grace_period=20
    )

    tuner = tune.Tuner(
        tune.with_resources(train_mnist, resources={"cpu": 2, "gpu": 0 }),
        tune_config=tune.TuneConfig(
            metric="mean_loss",
            mode="min",
            scheduler=sched,
            num_samples=100,
        ),
        run_config=air.RunConfig(
            name="exp",
            stop={"mean_accuracy": 0.99, "training_iteration": num_training_iterations},
        ),
         param_space={
            # "lr": tune.grid_search([0.001, 0.01,0.1]),
            # "beta1": tune.grid_search([0.6, 0.7,0.8,0.9]),
            # "beta2": tune.grid_search([ 0.7,0.8,0.9, 0.99]),
            # "batch_size": tune.grid_search([64, 128,256]),
            # "epochs": tune.grid_search([5, 10,15]),
            "lr": 0.01,
            "beta1": 0.8,
            "beta2": 0.9,
            "batch_size": tune.grid_search([64, 128,256]),
            "epochs": 5,
        },
    )
    global results
    results = tuner.fit()

    print("Best hyperparameters found were: ", results.get_best_result().config)


In [ ]:
from ray.tune.search import ConcurrencyLimiter
algo_b = BayesOptSearch(utility_kwargs={"kind": "ucb", "kappa": 4, "xi": 0.0})
algo_b = ConcurrencyLimiter(algo_b, max_concurrent=4)

def tune_baysian(num_training_iterations):
    sched = AsyncHyperBandScheduler(
        time_attr="training_iteration", max_t=400, grace_period=20
    )

    tuner = tune.Tuner(
        tune.with_resources(train_mnist, resources={"cpu": 2, "gpu": 0}),
        tune_config=tune.TuneConfig(
            metric="min_loss",
            mode="min",
            scheduler=sched,
            num_samples=50,
            search_alg = algo_b

        ),
        run_config=air.RunConfig(
            name="exp",
            stop={"mean_accuracy": 0.99, "training_iteration": num_training_iterations},
        ),
        param_space={
            "lr": tune.uniform(0, 1),
            "beta1": tune.uniform(0, 1),
            # "beta2": tune.uniform(0,1),
            "batch_size": 64,
            "epochs": 5,
        },
    )
    global results
    results = tuner.fit()


    print("Best hyperparameters found were: ", results.get_best_result().config)

In [ ]:

algo = NevergradSearch( optimizer=ng.optimizers.OnePlusOne)
algo = tune.search.ConcurrencyLimiter(algo, max_concurrent=4)


def tune_NevergradSearch(num_training_iterations):
    sched = AsyncHyperBandScheduler(
        time_attr="training_iteration", max_t=400, grace_period=20
    )

    tuner = tune.Tuner(
        tune.with_resources(train_mnist, resources={"cpu": 2, "gpu": 0}),
        tune_config=tune.TuneConfig(
            metric="mean_loss",
            mode="min",
            scheduler=sched,
            search_alg=algo,
            num_samples=50,
        ),
        run_config=air.RunConfig(
            name="NGAdam",
            stop={"mean_accuracy": 0.97, "training_iteration": num_training_iterations},
        ),
         param_space={
            "lr": tune.uniform(0, 1),
            "beta1": tune.uniform(0, 1),
            # "beta2": tune.uniform(0,1),
            "batch_size": 64,
            "epochs": 5,
        },
    )
    global results
    results = tuner.fit()

    print("Best hyperparameters found were: ", results.get_best_result().config)

In [ ]:
tune_grid(num_training_iterations=1)

(autoscaler +2m34s) Error: No available node types can fulfill resource request {'CPU': 2.0, 'GPU': 1.0}. Add suitable node types to this cluster to resolve this issue.


(pid=18399) 2023-04-27 10:49:48.512979: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Trial name,date,done,hostname,iterations_since_restore,mean_accuracy,mean_accuracy_val,mean_loss,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
train_mnist_37615_00000,2023-04-27_10-49-59,True,81a277c7d6e4,1,0.404,0.755,1.81105,172.28.0.12,18399,8.82904,8.82904,8.82904,1682592599,1,37615_00000
train_mnist_37615_00001,2023-04-27_10-50-10,True,81a277c7d6e4,1,0.2738,0.6437,2.16657,172.28.0.12,18399,10.8862,10.8862,10.8862,1682592610,1,37615_00001
train_mnist_37615_00002,2023-04-27_10-50-17,True,81a277c7d6e4,1,0.1642,0.4024,2.27392,172.28.0.12,18399,7.11649,7.11649,7.11649,1682592617,1,37615_00002
train_mnist_37615_00003,2023-04-27_10-50-29,True,81a277c7d6e4,1,0.4006,0.7735,1.80941,172.28.0.12,18399,11.2541,11.2541,11.2541,1682592629,1,37615_00003
train_mnist_37615_00004,2023-04-27_10-50-36,True,81a277c7d6e4,1,0.2406,0.6721,2.21956,172.28.0.12,18399,7.21844,7.21844,7.21844,1682592636,1,37615_00004
train_mnist_37615_00005,2023-04-27_10-50-48,True,81a277c7d6e4,1,0.1576,0.3231,2.27648,172.28.0.12,18399,11.6337,11.6337,11.6337,1682592648,1,37615_00005
train_mnist_37615_00006,2023-04-27_10-50-55,True,81a277c7d6e4,1,0.413,0.7692,1.81346,172.28.0.12,18399,7.68828,7.68828,7.68828,1682592655,1,37615_00006
train_mnist_37615_00007,2023-04-27_10-51-09,True,81a277c7d6e4,1,0.2614,0.5916,2.18623,172.28.0.12,18399,14.0282,14.0282,14.0282,1682592669,1,37615_00007
train_mnist_37615_00008,2023-04-27_10-51-17,True,81a277c7d6e4,1,0.1772,0.3744,2.26567,172.28.0.12,18399,7.40497,7.40497,7.40497,1682592677,1,37615_00008
train_mnist_37615_00009,2023-04-27_10-51-28,True,81a277c7d6e4,1,0.3918,0.769,1.87805,172.28.0.12,18399,10.9091,10.9091,10.9091,1682592688,1,37615_00009


(autoscaler +3m9s) Error: No available node types can fulfill resource request {'GPU': 1.0, 'CPU': 2.0}. Add suitable node types to this cluster to resolve this issue.
(autoscaler +3m44s) Error: No available node types can fulfill resource request {'CPU': 2.0, 'GPU': 1.0}. Add suitable node types to this cluster to resolve this issue.
(autoscaler +4m20s) Error: No available node types can fulfill resource request {'CPU': 2.0, 'GPU': 1.0}. Add suitable node types to this cluster to resolve this issue.
(autoscaler +4m55s) Error: No available node types can fulfill resource request {'GPU': 1.0, 'CPU': 2.0}. Add suitable node types to this cluster to resolve this issue.
(autoscaler +5m30s) Error: No available node types can fulfill resource request {'GPU': 1.0, 'CPU': 2.0}. Add suitable node types to this cluster to resolve this issue.
(autoscaler +6m5s) Error: No available node types can fulfill resource request {'CPU': 2.0, 'GPU': 1.0}. Add suitable node types to this cluster to resolve 

(autoscaler +58m10s) Error: No available node types can fulfill resource request {'GPU': 1.0, 'CPU': 2.0}. Add suitable node types to this cluster to resolve this issue.
(autoscaler +58m46s) Error: No available node types can fulfill resource request {'CPU': 2.0, 'GPU': 1.0}. Add suitable node types to this cluster to resolve this issue.
(autoscaler +59m21s) Error: No available node types can fulfill resource request {'CPU': 2.0, 'GPU': 1.0}. Add suitable node types to this cluster to resolve this issue.


In [ ]:
print("Best hyperparameters found were: ", results.get_best_result())

In [ ]:
dataframe_results = results.get_dataframe()

In [ ]:
# Sort the dataframe by the "loss" column in ascending order
# dataframe_results = dataframe_results.sort_values("mean_accuracy")

# Retrieve the parameters of the best performing trial(s)
best_trials = dataframe_results.nlargest(5, 'mean_accuracy')

# # Print a table with the parameters of the best trial(s)
# print(f"Best trial with loss {best_trials['mean_accuracy']}:")
# for param, value in best_trials.items():
#     print(f"{param}: {value}")

print(best_trials)

In [ ]:
dataframe_results.head()

In [ ]:
import matplotlib.pyplot as plt

# create the axis object
ax = plt.gca()

# plot the data
dataframe_results.plot(kind='line', y='config/eta1', ax=ax, use_index=True)
# dataframe_results.plot(kind='line', y='config/beta2', color='red', ax=ax, use_index=True)
dataframe_results.plot(kind='line', y='config/lr', color='green', ax=ax, use_index=True)

# set the title and axis labels
ax.set_title('Hyperparameter values : Nevergrad with SDG ')
ax.set_xlabel('Iterations')
ax.set_ylabel('Values')

# show the plot
plt.show()

In [ ]:
ax = plt.gca()
dataframe_results.plot(kind='line',y='mean_accuracy',ax=ax, use_index=True)
dataframe_results.plot(kind='line',y='mean_accuracy_val', color='red', ax=ax, use_index=True)
#dataframe_results.plot(kind='line',y='config/lr', color='green', ax=ax, use_index=True)
#dataframe_results.plot(kind='line',y='mean_loss', color='green', ax=ax, use_index=True)
# dataframe_results.plot(kind='line',y='mean_accuracy', color='black', ax=ax, use_index=True)

ax.set_title('Accuracy values : Nevergrad with SDG')
ax.set_xlabel('Iterations')
ax.set_ylabel('Values')

# show the plot
plt.show()